# Práctica 3 Sistemas inteligentes Segmentación de imágenes mediante técnicas de agrupamiento
## Hecho por César Rodríguez Villagrá

En esta práctica se hace la implementación de la segmentación de imágenes mediante técnicas de agrupamiento.
Las 3 técnicas que se van a utilizar son:
- K-Means
- Fuzzy C-Means (FCM)
- Mixtura de Gausianas (MixGaus)

Los resultados se encuentran también en la carpeta fotos, en el que cada algoritmo tiene su carpeta con los resultados obtenidos.

## Implementación

### Importaciones necesarias

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from skimage import io
from sklearn.cluster import KMeans
import skfuzzy as fuzz
from sklearn.mixture import GaussianMixture
import os

### Funciones de aplicación de cada algoritmo
Se definen las funciones con los parámetros que se van a aplicar en pasos posteriores.

In [2]:
def aplicarKMeans(imagen, numero_clusters: int, max_iteraciones: int):
    """Función para aplicar el algoritmo de K-Means sobre una imagen

    Args:
        imagen: la imagen a procesar
        numero_clusters (int): el número de clusters necesarios
        max_iteraciones (int): las máximas iteraciones para el algoritmo

    Returns:
        imagen: la imagen ya procesada por el método
    """
    filas, columnas, canales = imagen.shape
    imagen_reshape = imagen.reshape((filas * columnas, canales))

    kmeans = KMeans(n_clusters=numero_clusters,
                    max_iter=max_iteraciones, n_init='auto')
    kmeans.fit(imagen_reshape)

    etiquetas = kmeans.predict(imagen_reshape)
    centros = kmeans.cluster_centers_
    imagen_segmentada = centros[etiquetas].reshape(imagen.shape)

    return imagen_segmentada.astype(np.uint8)


def aplicarFCM(imagen, numero_grupos: int, max_iteraciones: int, m: float, err: float):
    """Función para aplicar el algoritmo de Fuzzy C-Means sobre una imagen

    Args:
        imagen: la imagen a procesar
        numero_grupos (int): el número de grupos a aplicar
        max_iteraciones (int): las máximas iteraciones para el algoritmo
        m (float): valor de fuzzifier
        err (float): error aceptable al realizar el proceso de minimización

    Returns:
        imagen: la imagen ya procesada por el método
    """
    filas, columnas, canales = imagen.shape
    imagen_reshape = imagen.reshape((filas * columnas, canales))

    cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(
        imagen_reshape.T, numero_grupos, m, error=err, maxiter=max_iteraciones, init=None)

    etiquetas = np.argmax(u, axis=0)
    imagen_segmentada = etiquetas.reshape((filas, columnas))

    return imagen_segmentada.astype(np.uint8)


def aplicarMixGaus(imagen, num_gaus: int, tipo_covarianza, max_iteraciones: int):
    """Función para aplicar el algoritmo de Mixtura de Gausianas sobre una imagen

    Args:
        imagen: la imagen a procesar
        num_gaus (int): el número de gausianas que queremos aplicar
        tipo_covarianza (str): tipo de la matriz de covarianza asociada a cada gausiana
        max_iteraciones (int): las máximas iteraciones para el algoritmo

    Returns:
        imagen: la imagen ya procesada por el método
    """
    filas, columnas, canales = imagen.shape
    imagen_reshape = imagen.reshape((filas * columnas, canales))

    mg = GaussianMixture(n_components=num_gaus, covariance_type=tipo_covarianza,
                         max_iter=max_iteraciones)

    mg.fit(imagen_reshape)
    etiquetas = mg.predict(imagen_reshape)
    imagen_segmentada = etiquetas.reshape((filas, columnas))

    return imagen_segmentada.astype(np.uint8)

### K-Means
En este algoritmo se va a variar varios parámetros en la aplicación del algoritmo, en este caso:
- El **número de grupos** entre los que se va a segmentar la imagen.
- El **número máximo de iteraciones** que puede realizar el algortimo para encontrar el resultado.

En este caso se va a realizar las pruebas con valores de cluster entre 2 y 6, ya que entre ese rango contiene los distintos tipos de colores más predominantes en la imagen, para que los pueda separar.
También se realizará con unas iteraciones máximas entre 100 y 500, valores equilibrados entre unos resultados aceptables y un tiempo de cálculo no muy elevado, ya que este valor es el número máximo de ajustes que hace a los puntos, si este número es menor al óptimo puede generar un resultado poco óptimo. Este número es un criterio para parar de ejecutar del algoritmo, otro es cuando entre iteraciones no cambian mucho los resultados.

In [ ]:
ruta = "./Fotos/KMeans/"
if not os.path.exists(ruta):
    os.makedirs(ruta)

imageA = 'Colorful'
imageB = 'Keyboard'
imageC = 'PoolBar'

imagesName = [imageA, imageB, imageC]

for imagenN in imagesName:
    imagen = io.imread(imagenN+'.jpg')
    plt.figure()
    plt.imshow(imagen)
    plt.title('Imagen base: '+imagenN+' para aplicar K-Means')
    plt.axis('off')

    n_clusters = [2, 3, 4, 5, 6]
    nmax_iteraciones = [100, 200, 300, 400, 500]

    fig, ax = plt.subplots(nrows=len(n_clusters), ncols=len(
        nmax_iteraciones), figsize=(21, 15))

    for fila in range(len(n_clusters)):
        for columna in range(len(nmax_iteraciones)):
            ax[fila][columna].imshow(aplicarKMeans(
                imagen, n_clusters[fila], nmax_iteraciones[columna]))
            ax[fila][columna].set_title(str(n_clusters[fila]) + ' clusters con ' +
                                        str(nmax_iteraciones[columna]) + ' itermáx')
            ax[fila][columna].axis('off')
    fig.tight_layout()
    plt.savefig(ruta+"RepresentacionKMeans"+imagenN+".jpg")
    plt.show()

Como se puede ver en las 3 imágenes procesadas, cuantos más grupos se apliquen mejor hace la diferenciación. Por ejemplo la primera imagen a partir de 5 grupos no tendría cambio, ya que es una imagen con unos colores muy marcados y con muy poco ruido o incluso sin ruido.
En la segunda imagen logra también un muy buen clusterizado, en cambio en la tercera imagen al tener más elementos diferenciables con 6 clusters se puede quedar corto, dependiendo de la precisión que queremos tener, aunque con 5-6 clusters ya se puede ver una aceptable agrupación de los elementos. Dependiendo de la cantidad de grupos que se quiera manejar hará la separación entre más elementos o menos.

### Fuzzy C-Means
En este algoritmo se va a variar varios parámetros en la aplicación del algoritmo, en este caso:
- El **número de grupos**.
- El **número máximo de iteraciones** que puede realizar el algortimo para encontrar el resultado.
- El parámetro **m** o **"fuzzifier"**, que controla la cantidad de solapamiento de un punto sobre distintos grupos, a un valor mayor, mayor será la difusión.
- El **error** indica el valor que puede haber como máximo de diferencia entre 2 iteraciones para aceptar esa iteración como solución, a menor valor se obtiene mayor precisión pero necesitará más iteraciones para llegar a ese resultado.

En este caso se va a ejecutar el algoritmo con2, 4 y 6 grupos, con 3, 30 y 100 iteraciones máximas, con valores m de 1.5 y 2 y con errores de 0.005 y 0.05.
En este caso se han reducido los valores de los parámetros debido a la gran cantidad de tiempo que tarda en ejecutar.
Los valores con más impacto en el tiempo de ejecución son el número máximo de iteraciones y el error, ya que son los parametros de convergencia que permiten parar antes al alagoritmo.
Si el valor m es de valor 1 el resultado sería el mismo que el de k-means, porque no habría "difusión" entre grupos, por lo que he optado en utilizar valores entre 1 y 2.

In [ ]:
ruta = "./Fotos/FCM/"
if not os.path.exists(ruta):
    os.makedirs(ruta)

imageA = 'Colorful'
imageB = 'Keyboard'
imageC = 'PoolBar'

imagesName = [imageA, imageB, imageC]

for imagenN in imagesName:
    imagen = io.imread(imagenN+'.jpg')
    plt.figure()
    plt.imshow(imagen)
    plt.title('Imagen base: '+imagenN+' para aplicar Fuzzy C-Means')
    plt.axis('off')

    n_grupos = [2, 4, 6]
    nmax_iteraciones = [3, 30, 100]
    m_valor = [1.5, 2.0]
    error_valor = [0.005, 0.05]
    for m in m_valor:
        for error in error_valor:
            fig, ax = plt.subplots(nrows=len(n_grupos), ncols=len(
                nmax_iteraciones), figsize=(21, 15))

            for fila in range(len(n_grupos)):
                for columna in range(len(nmax_iteraciones)):
                    ax[fila][columna].imshow(aplicarFCM(
                        imagen, n_grupos[fila], nmax_iteraciones[columna], m, error))
                    ax[fila][columna].set_title(str(n_grupos[fila]) + ' clusters, ' +
                                                str(nmax_iteraciones[columna]) + ' itermáx, ' +
                                                str(m) + ' m-valor, ' +
                                                str(error) + ' error')
                    ax[fila][columna].axis('off')
            fig.tight_layout()
            plt.savefig(ruta+"RepresentacionFCM"+imagenN + 'm' +
                        str(m)+'error'+str(error)+".jpg")
            plt.show()

Debido a la cantidad de parámetros que se han comparado, al número máximo de iteraciones y de error y probablemente la implementación del algortimo ha tenido un tiempo de ejecución elevado, de 1 hora aproximadamente.

En la primera imagen se hace una división parecida al k-means pero difusa y menos exacta. El cambio del parámetro m no influye mucho entre 1.5 y 2, aunque tiene algunos cambios visibles.

En la segunda imagen no se aprecia/realiza la clusterización de los distintos colores de las letras(solo se puede diferenciar facilmente con 6 clusteres, 100 it, 1.5 de m y 0.05 de error), aunque nos proporciona detalles de las teclas, en el que en la imagen orginal parecen todas las teclas "iguales", pero después del procesado se puede apreciar las sombras y el ruido.

En la última imagen se puede apreciar mejor las zonas que pertenecen a varios grupos, como puede ver las sombras y las luces.

En todas las imágenes se puede apreciar que cuando se tiene 3 iteraciones máximas es da un resultado impreciso.

### Mixtura de Gausianas
En este algoritmo se va a variar varios parámetros en la aplicación del algoritmo, en este caso:
- El **número de gausianas**.
- El **número máximo de iteraciones** que puede realizar el algortimo para encontrar el resultado.
- **Tipo de covarianza de la matriz**, en este caso o diag o full

Se va a realizar la ejecución con 3, 5 y 6 gausianas, con iteraciones entre 75 y 150, con matrices de covarianza diagonal y completa.

In [ ]:
ruta = "./Fotos/MixGaus/"
if not os.path.exists(ruta):
    os.makedirs(ruta)

imageA = 'Colorful'
imageB = 'Keyboard'
imageC = 'PoolBar'

imagesName = [imageA, imageB, imageC]

for imagenN in imagesName:
    imagen = io.imread(imagenN+'.jpg')
    plt.figure()
    plt.imshow(imagen)
    plt.title('Imagen base: '+imagenN+' para aplicar Mixtura de gausianas')
    plt.axis('off')

    num_gaus = [3, 5, 6]
    nmax_iteraciones = [75, 100, 150]
    tipo_covarianza = ['diag', 'full']

    for tipo in tipo_covarianza:
        fig, ax = plt.subplots(nrows=len(num_gaus), ncols=len(
            nmax_iteraciones), figsize=(21, 15))

        for fila in range(len(num_gaus)):
            for columna in range(len(nmax_iteraciones)):
                ax[fila][columna].imshow(aplicarMixGaus(
                    imagen, num_gaus[fila], tipo, nmax_iteraciones[columna]))
                ax[fila][columna].set_title(str(num_gaus[fila]) + ' gausianas con ' +
                                            str(nmax_iteraciones[columna]) + ' itermáx '+tipo)
                ax[fila][columna].axis('off')
        fig.tight_layout()
        plt.savefig(ruta+"RepresentacionMixGaus"+tipo+imagenN+".jpg")
        plt.show()

En la primera imagen con el tipo de covarianza diagonal obtiene resultados alejados a la imagen real, algo que soluciona el modo full, en el que hace la mejor la diferencicación de la imagen.

En cambio en la segunda imagen muestra resultados con menos ruido que el modo full.

En la tercera imagen la distinción entre objetos es más inexacta, aunque realiza la distinción de las zonas más predominantes.

Entre los distintos tipos de covarianza de las matrices se puede observar un cambio entre ellos, pero dependiendo de la imagen produce mejores o peores resultados, en otros casos se tiene que eveluar qué tipo es mejor en ese caso.
Otra cosa reseñable es que como ya se ha dicho anteriormente cuantas menos iteraciones máximas tengan pueden generar resultados más inexactos, si se quiere mejores resultados se recomiendaría subir este valor al máximo posible computacionalmente con un tiempo aceptable.

## Conclusiones

En los 3 métodos vistos en esta práctica se puede apreciar las diferencias entre ellos y los distintos ajustes que se pueden hacer a cada uno.
Para cada imagen que se quiera segmentar habría que hacer un miniestudio para ver qué parámetros son los más adecuados para los reultados que queramos, incluyendo la selección del método más preciso para ese caso y los distintos parámetros.
Otro factor a condiderar es el tipo de la imagen, si tiene sombras, ruido, luces, distintos tipos de objetos etc.

En este caso, lo más reseñable es que en la primera imagen para poder separar los colores con más precisión ha sido kmeans, en la segunda imagen dependiendo la información que se quiera extraer, se podrían utilizar los 3 métodos. Y en la tercera imagen lo más recomendable sería utilizar FCM. Otro aspecto clave es la identificación de la cantidad de los grupos/clusters/gausianas que según el caso debe variar.

Como conclusión final, en esta práctica se puede ver las distintas capacidades que pueden tener los métodos vistos, y queda a criterio del programador cuál aplicar y con qué parámetros a cada imagen específica, dependiendo de la información que se quiera extraer de ella y los recursos disponibles (tiempo y velocidad de computación principalmente).